In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
import contextily as cx
import datetime as dt
import json
from joblib import Parallel, delayed

from functions_file import *
print(testfunction(4))

## Process data
First, reformat data from MSs and LDs for all polygons. Then gather and save the data in one file.

### 1. Reformat MS and LD data

**Key settings for reformatting**

In [ ]:
polygons = pd.read_csv('../data/polygons11.csv')
polygons = polygons.drop([4])
polygons['polygon'] = polygons.name
poly_cols = ['polygon','lanes','direction','busstops','seplane','length','complexity','road_rank']

savefactors = {}
scalefactorsfile = 'scalefactors_bypolygon.pkl' # 'scalefactors_bypolygon_bymode.pkl'

# LD and MS data should have same resampling
resample = '30s' 
window = '3T' # 3min
polygon_names = polygons.name.values

**Mobile sensor data**

In [ ]:
#################
# Resample, scale and gather data for mobile sensor modes
#################
save = 'off'

for polygon_name in polygon_names:
    
    POLYGON = get_polygon(polygon_name,polygons)
    # 1. load 'regular' and 'car' data
    # 1.a. file 1 - regular
    nonresampled = import_sensor_data(POLYGON['name'],'MS')
    modes        = set(nonresampled['mode'])
    resampled    = sensor_resample_window(nonresampled,modes,resample=resample,window=window,POLYGON=POLYGON,plot='off',filna=False)
    #display(nonresampled[nonresampled['mode']=='all'].head())
    #display(resampled[resampled['mode']=='all'].head())
    # 1.b. file 2 - car penetration rates
    nonresampled_cars = import_sensor_data(POLYGON['name'],'MS',cars='Yes')
    modes_cars        = set(nonresampled_cars['mode'])
    resampled_cars    = sensor_resample_window(nonresampled_cars,modes_cars,resample=resample,window=window,POLYGON=POLYGON,plot='off',filna=False)
    # 1.c. combine data
    data = pd.concat([resampled,resampled_cars])
    
    # 2. normalize/scale by link
    cols       = ['densities','flows']
    xx         = data[data['mode']=='all'][cols]
    means      = xx.mean()
    data[cols] = data[cols]/xx.mean() 
    interval   = data[(data['mode']=='all')&(data.densities>0.2)&(data.densities<2)]
    slope      = interval.flows.mean()/interval.densities.mean()
    data.flows = data.flows/slope
    savefactors[polygon_name+'_MS'] = [1,slope,means[0],means[1]]
    
    # 3. reformat - create a df per mode, rename columns, merge
    result = []
    for mode in set(data['mode']):
        # can exclude exp_id - not needed & issues due to diff. experiment start/end times
        tmp = data[data['mode']==mode][['exp_id','polygon','DOW','times','speeds','densities','flows']]
        tmp = tmp.rename(columns={'speeds':'v_%s'%(mode),'densities':'k_%s'%(mode),'flows':'q_%s'%(mode)})
        if (len(result)<1):
            result = tmp
        else:
            result = pd.merge(result, tmp, on=['polygon','DOW','times','exp_id'],how='outer')
    
    # 4. add polygon information
    result = pd.merge(result, polygons[poly_cols], how='outer', on=['polygon'])
    display(result.head(2))
    
    # 5. save
    if save=='on':
        result.to_pickle('../output/data_processed/processed_data_MS_%s.pkl'%(POLYGON['name']))
        print('Saved %s.'%(polygon_name))
    print('Length:',len(result))


**Loop detector data**

In [ ]:
#################
# Resampe, scale and gather data for 'all' using loop detector approach
#################
save = 'off'

for polygon_name in polygon_names:
    
    POLYGON = get_polygon(polygon_name,polygons)
    # 1. load 'all' from 'regular' data
    nonresampled = import_sensor_data(POLYGON['name'],sensor='LD')
    modes        = set(nonresampled['mode'])
    resampled    = sensor_resample_window(nonresampled,modes,resample=resample,window=window,POLYGON=POLYGON,plot='off')
    data         = resampled
    
    # 2. normalize/scale by link
    xx = data[data['mode']=='all'][['densities','flows']]
    means    = xx.mean()
    data[['densities','flows']] = data[['densities','flows']]/xx.mean() # dont need exp_id or speeds
    interval = data[(data['mode']=='all')&(data.densities>0.2)&(data.densities<2)]
    slope    = interval.flows.mean()/interval.densities.mean()
    savefactors[polygon_name+'_LD'] = [1,slope,means[0],means[1]]
    
    # 3. reformat - create a df per mode, rename columns, merge
    result = []
    for mode in set(data['mode']):
        tmp = data[data['mode']==mode][['exp_id','polygon','DOW','times','speeds','densities','flows']]
        tmp = tmp.rename(columns={'speeds':'v_%s'%(mode),'densities':'k_%s'%(mode),'flows':'q_%s'%(mode)})
        if (len(result)<1):
            result = tmp
        else:
            result = pd.merge(result, tmp, on=['polygon','DOW','times','exp_id'],how='outer')
    
    # 4. add polygon information
    result = pd.merge(result, polygons[poly_cols], how='outer', on=['polygon'])
    display(result.head(2))
    
    # 5. save
    if save=='on':
        result.to_pickle('../output/data_processed/processed_data_LD_%s.pkl'%(POLYGON['name'])) # index=False#with parked
        print('Saved.')
    print('Length:',len(result))

#### Save scalefactors

In [ ]:
# save scalefactors
with open(scalefactorsfile, 'wb') as f:
    pickle.dump(savefactors, f)    
# to read
with open(scalefactorsfile, 'rb') as f:
    scalefactors = pickle.load(f)

### 2. Gather all data

In [ ]:
save = 'off'

# 1. load and merge data
all_MS = pd.concat( [pd.read_pickle('../output/data_processed/processed_data_MS_%s.pkl'%(p)) for p in polygons.name.values] )
all_LD = pd.concat( [pd.read_pickle('../output/data_processed/processed_data_LD_%s.pkl'%(p)) for p in polygons.name.values] )
vqk_cols_MS = [col for col in all_MS if col.startswith(('q_','v_','k_'))]
vqk_cols_LD = [col for col in all_LD if col.startswith(('q_','v_','k_'))]
all_MS.columns = [a+'_MS' if a in vqk_cols_MS else a for a in all_MS.columns]
all_LD.columns = [a+'_LD' if a in vqk_cols_LD else a for a in all_LD.columns]
non_vqk_cols = [col for col in all_MS if not col.startswith(('q_','v_','k_'))]
all_data = pd.merge(all_MS, all_LD, how='outer',on=non_vqk_cols,suffixes = ("_MS","_LD"))

# 2.a. set tiny values to zero
vqk_cols = [col for col in all_data if col.startswith(('q_','v_','k_'))]
for c in vqk_cols:
    tmp = all_data[c]
    tmp = [0 if a<1e-10 else a for a in tmp]
    all_data[c] = tmp
# 2.b. drop where ALL of [v,q,k] in [0,nan] --> either no traffic or no footage because in between recordings
leninit = len(all_data)
all_data = all_data.loc[~(all_data[vqk_cols].isin([0,np.nan])).all(axis=1)]
print('%.1f%% removed due to ambiguity (no traffic or no data?). New length: %s.'%(100*(1-len(all_data)/leninit),len(all_data)))

# 3. replace q and k NaNs with 0s
all_q_k_cols = [col for col in all_data if col.startswith(('q_','k_'))]
all_data[all_q_k_cols] = all_data[all_q_k_cols].fillna(0)
print('\nNaNs:\n',all_data[[col for col in all_data if col.startswith(('q_','v_','k_'))]].isna().sum())

# 4. save
if save=='on':
    #all_data.to_pickle('../output/processed_data/processed_data_all_bypolygonandmode.pkl') # index=False
    all_data.to_pickle('../output/data_processed/processed_data_all_bypolygon.pkl') # index=False
    print('Saved.')
display(all_data.head(3))
print('Length:',len(all_data))

### 3. Check some stuff

#### Check q-k plot of normalized data

In [ ]:
data = all_data 

# allMS data
plt.figure(figsize=(9,6))
for i in [0,1,2,3,5,6,7,8,9,10,11]:
    subset = data[data.polygon=='polygon_r%s'%i]
    plt.scatter(subset.k_all_MS,subset.q_all_MS,s=8,alpha=0.3,label=subset.polygon.values[0])

plt.title('All mobile sensor data, interval length: %s, rolling window: %s'%(resample,window))
plt.xlabel('Density k (scale converted)'); plt.ylabel('Flow q (scale converted)'); plt.legend()
#plt.savefig('All_MS_scaled.png')
plt.show()

# allLD data
plt.figure(figsize=(9,6))
for i in [0,1,2,3,5,6,7,8,9,10,11]:
    subset = data[data.polygon=='polygon_r%s'%i]
    plt.scatter(subset.k_all_LD,subset.q_all_LD,s=8,alpha=0.3,label=subset.polygon.values[0])

plt.title('All virtual loop detector data')
plt.xlabel('Density k (scale converted)');plt.ylabel('Flow q (scale converted)'); plt.legend()

plt.show()
print(len(all_data))